# Analysis of a Single Shot - Errors on the IV Characteristic 

This is a notebook for analysing a single shot from second round magnum data and looking closely at the distribution of values around the averaged IV characteristic

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/Data/external/magnum/'
path_to_analysed_datasets = 'analysed_2/*nc'
os.chdir(path_to_datasets)

analysed_dataset_fns = glob.glob(path_to_analysed_datasets)
analysed_dataset_fns.sort()

analysed_infos = []

for i, anal_ds in enumerate(analysed_dataset_fns):
    match = re.search("\/a{1}([0-9]{3})_([0-9]{3})_([0-9]{19})\.nc", anal_ds)
    shot_index = int(match.group(1))
    adc_index = int(match.group(2))
    shot_timestamp = int(match.group(3))
    shot_time = ut.human_datetime_str(int(match.group(3)))
    
    ds = xr.open_dataset(anal_ds)
    time_len = len(ds['time'])
    sweep_len = len(ds['sweep'])
    
    analysed_infos.append({
        'adc_index':adc_index, 
        'shot_number':shot_index, 
        'shot_timestamp':shot_timestamp, 
        'shot_time':shot_time, 
        'filename':anal_ds,
        'time_len': time_len,
        'sweep_len': sweep_len
    })

analysed_infos_df = pd.DataFrame(analysed_infos).set_index('adc_index')

In [3]:
adc_index = 329

In [4]:
shot_number = 337

In [13]:
# analysed_infos_df.loc[adc_index]
analysed_infos_df

,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
adc_index,,,,,,
5,analysed_2/a003_005_6696074692831397888.nc,3,2019-05-28 14:49:40,6696074692831397888,499,50
6,analysed_2/a004_006_6696076438102672384.nc,4,2019-05-28 14:56:27,6696076438102672384,798,50
7,analysed_2/a005_007_6696076936923400192.nc,5,2019-05-28 14:58:23,6696076936923400192,798,50
8,analysed_2/a006_008_6696082451310700544.nc,6,2019-05-28 15:19:47,6696082451310700544,799,50
9,analysed_2/a007_009_6696082985479260160.nc,7,2019-05-28 15:21:51,6696082985479260160,1999,50
10,analysed_2/a008_010_6696084010909390848.nc,8,2019-05-28 15:25:50,6696084010909390848,798,50
11,analysed_2/a009_011_6696084446035357696.nc,9,2019-05-28 15:27:31,6696084446035357696,799,50
12,analysed_2/a010_012_6696085352966572032.nc,10,2019-05-28 15:31:02,6696085352966572032,799,50
13,analysed_2/a011_013_6696085746216338432.nc,11,2019-05-28 15:32:34,6696085746216338432,799,50


In [6]:
# Get row from shot_number instead of ADC index
shot_oi = analysed_infos_df.loc[analysed_infos_df['shot_number'] == shot_number]
shot_oi['filename'].values

array(['analysed_2/a337_329_6699048922136187904.nc'], dtype=object)

In [7]:
analysis_ds = xr.open_dataset(shot_oi['filename'].values[0])
metadata_ds = xr.open_dataset('all_meta_data.nc').sel(shot_number=shot_number).load()
print(analysis_ds)
metadata_ds

<xarray.Dataset>
Dimensions:     (direction: 2, probe: 2, sweep: 2198, time: 50)
Coordinates:
  * time        (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0047 0.0048 0.0049
  * direction   (direction) object 'up' 'down'
  * probe       (probe) object 'S' 'L'
Dimensions without coordinates: sweep
Data variables:
    voltage     (probe, direction, sweep, time) float64 ...
    current     (probe, direction, sweep, time) float64 ...
    shot_time   (probe, direction, sweep, time) float64 ...
    start_time  (probe, direction, sweep) float64 ...


<xarray.Dataset>
Dimensions:                 (ts_radial_pos: 46)
Coordinates:
    shot_number             int32 337
    ts_number               float64 404.0
    ts_timestamp            float64 6.699e+18
    ts_time                 datetime64[ns] 2019-06-05T15:11:12.409731
    adc_index               float64 329.0
    adc_time                datetime64[ns] 2019-06-05T15:11:12
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            object '2019-06-05 16h 11m 34s TT_06699048922136188177.adc'
    ts_density              (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_temperature          (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_density            (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_temperature        (ts_radial_pos) float64 nan nan nan ... nan nan nan
    adc_folder              object '2019-06-05_Leland/'
    adc_calibration_index   object '272'
    adc_4_probe    

## Do some analysis of shot

In [8]:
probe = 'S'

In [9]:
fig, ax = plt.subplots(3)
shot_ds = analysis_ds.sel(probe=probe)

for sweep in shot_ds['sweep'].values[::2]:
    sweep_ds = shot_ds.sel(sweep=sweep, direction='up')
    ax[0].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, color='silver')
    sweep_ds = shot_ds.sel(sweep=sweep, direction='down')
    ax[1].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, color='silver')
    
sweep_avg_ds = shot_ds.mean('sweep')
for i, direction in enumerate(['up', 'down']):
    ax[i].plot(sweep_avg_ds.sel(direction=direction)['voltage'].values, sweep_avg_ds.sel(direction=direction)['current'].values, 'k', label='Average')
    ax[2].plot(sweep_avg_ds.sel(direction=direction)['voltage'].values, sweep_avg_ds.sel(direction=direction)['current'].values, label=f'{direction} avg.')
plt.show()

In [10]:
zeroed_ds = shot_ds - sweep_avg_ds
bins = np.linspace(-0.06,0.06,200)

plt.figure()
for time in zeroed_ds['time'].values:
    direction = 'up'
    time_ds = zeroed_ds.sel(time=time, direction=direction)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['current'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bins, hist)
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
    
plt.show()

In [11]:
bins = np.linspace(-0.1,0.1,200)

plt.figure()
for time in zeroed_ds['time'].values:
    direction = 'up'
    time_ds = zeroed_ds.sel(time=time, direction=direction)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['voltage'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bins, hist)
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
    
plt.show()